In [ ]:
if True:
    import pandas as pd
    from datasets import Dataset
    from transformers import DistilBertTokenizer

    X_train = pd.read_csv("../data/Train_Test_splits/X_train_50proc_trunc_pad.csv")
    X_test = pd.read_csv("../data/Train_Test_splits/X_test_50proc_trunc_pad.csv")
    y_train = pd.read_csv("../data/Train_Test_splits/y_train_50proc.csv")
    y_test = pd.read_csv("../data/Train_Test_splits/y_test_50proc.csv")

    print(X_train.columns)
    print(X_test.columns)
    print(y_train.columns)
    print(y_test.columns)

# TF #

In [ ]:
if False:
    from datasets import load_dataset
    from transformers import TFAutoModelForSequenceClassification, DistilBertTokenizer
    import tensorflow as tf
    from sklearn.metrics import accuracy_score
    import numpy as np

    if not tf.config.list_physical_devices("GPU"):
        print("Warning: No GPU found. Performance may be slower.")

    tokenizer = DistilBertTokenizer.from_pretrained(
        "distilbert-base-uncased-finetuned-sst-2-english"
    )
    model = TFAutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased-finetuned-sst-2-english"
    )

    dataset = load_dataset("imdb")
    test_dataset = dataset["test"].select(range(100))

    def tokenize_function(examples):
        return tokenizer(
            examples["text"], padding="max_length", truncation=True, max_length=512
        )

    test_dataset = test_dataset.map(tokenize_function, batched=True)

    test_tf_dataset = tf.data.Dataset.from_tensor_slices(
        (
            np.array(test_dataset["input_ids"]),
            np.array(test_dataset["attention_mask"]),
            np.array(test_dataset["label"]),
        )
    ).batch(8)

    @tf.function
    def predict_step(input_ids, attention_mask):
        inputs = {"input_ids": input_ids, "attention_mask": attention_mask}
        logits = model(inputs, training=False).logits
        return np.argmax(logits.numpy(), axis=1)

    predictions = []
    labels = []

    for input_ids, attention_mask, batch_labels in test_tf_dataset:
        preds = predict_step(input_ids, attention_mask)
        predictions.extend(preds)
        labels.extend(batch_labels.numpy())

    accuracy = accuracy_score(labels, predictions)
    print(f"Accuracy: {accuracy}")

In [7]:
if False:
    import tensorflow as tf
    print(tf.config.list_physical_devices("GPU"))

# Torch #

In [ ]:
if True:
    from datasets import load_dataset
    from transformers import AutoModelForSequenceClassification, DistilBertTokenizer
    import torch
    from sklearn.metrics import accuracy_score
    import numpy as np

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    tokenizer = DistilBertTokenizer.from_pretrained(
        "distilbert-base-uncased-finetuned-sst-2-english"
    )
    model = AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased-finetuned-sst-2-english"
    ).to(device)

    dataset = load_dataset("imdb")
    test_dataset = dataset["test"].select(range(100))

    def tokenize_function(examples):
        return tokenizer(
            examples["text"], padding="max_length", truncation=True, max_length=512
        )

    test_dataset = test_dataset.map(tokenize_function, batched=True)

    inputs = torch.tensor(test_dataset["input_ids"]).to(device)
    attention_mask = torch.tensor(test_dataset["attention_mask"]).to(device)
    labels = torch.tensor(test_dataset["label"]).to(device)

    predictions = []
    true_labels = []

    with torch.no_grad():
        for i in range(0, len(inputs), 8):
            batch_input_ids = inputs[i : i + 8]
            batch_attention_mask = attention_mask[i : i + 8]
            batch_labels = labels[i : i + 8]
            outputs = model(batch_input_ids, attention_mask=batch_attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(batch_labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    print(f"Accuracy: {accuracy}")

In [ ]:
if True:
    import tensorflow as tf

    print(tf.config.list_physical_devices("GPU"))
    devices = tf.config.list_physical_devices()
    print(devices)

In [ ]:
if True:
    import tensorflow as tf

    device = tf.config.list_physical_devices("GPU")

    device_string = "GPU is available!" if device else "GPU is NOT available."
    print()
    print(device_string)
    print()

    # Somehow this takes unbelievably long time...
    with tf.device("/GPU:0"):
        a = tf.random.normal([10, 10])
        b = tf.random.normal([10, 10])
        c = tf.matmul(a, b)

        print("Matrix multiplication done on GPU!")